# 필요한 모듈 불러오기

In [ ]:
# https://github.com/pytorch/examples/blob/master/mnist/main.py
from __future__ import print_function # python 2, 3 어떤 버전을 돌리느냐에 상관없이 python 3의 print 함수 기능을 사용가능
from torch import nn, optim, cuda # cuda : GPU 사용과 관련된 모듈. is_available() 사용하여 컴퓨터가 CUDA 돌릴 수 있는지 판단가능
from torch.utils import data # Dataloader 사용하기 위해 불러옴

# datasets : 여러가지 대표적인 데이터셋을 만들 수 있도록 해줌. 여기서는 MNIST
# transforms : image transformation 에 사용하는 모듈
from torchvision import datasets, transforms 

# nn의 존재하는 다양한 함수가 존재하는 functional 불러옴
import torch.nn.functional as F

# 날짜와 시간 처리를 해주는 모듈
import time

## batch size 설정 및 GPU 사용 기능 활성화

In [ ]:
# Training settings
batch_size = 64

# cuda(GPU) 사용 가능 -> cuda.is_availabe() == 'True'
# 불가능 -> cpu 사용
device = 'cuda' if cuda.is_available() else 'cpu' 
print(f'Training MNIST Model on {device}\n{"=" * 44}')

## custom dataset 생성 - 훈련용과 테스트용 분리 - MNIST Dataset 을 가져온다 torchvision.datasets.MNIST

In [ ]:
# MNIST Dataset
train_dataset = datasets.MNIST(root='./mnist_data/', # dataset의 root directory : MNIST/processed/training.pt 와 MNIST/processed/test.pt가 존재
                               train=True, # training.pt으로부터 dataset 생성
                               transform=transforms.ToTensor(), # PIL image 를 Tensor type 으로 변환
                               download=True) # True -> 인터넷으로부터 dataset 다운로드 받고 root directory에 넣음. 
                                                # 이미 다운로드 되어 있으면 다운로드 안함

test_dataset = datasets.MNIST(root='./mnist_data/',
                              train=False, # test.pt으로부터 dataset 생성
                              transform=transforms.ToTensor())

## custom dataloader 생성 -훈련용 과 테스트용 분리

In [ ]:
# Data Loader (Input Pipeline)
train_loader = data.DataLoader(dataset=train_dataset, # train_loader: train_dataset 이용한 custom dataloader
                                           batch_size=batch_size, # batch_size 64
                                           shuffle=True) # 무작위로 섞어줌

test_loader = data.DataLoader(dataset=test_dataset, # test_loader: test_dataset 이용한 custom dataloader
                                          batch_size=batch_size, # batch_size 64
                                          shuffle=False) # 섞지 않음 -> 순서대로 테스트

## 클래스 Net 생성

### view : tensor의 크기(size)나 모양(shape)을 변경하고자 할 때 사용

### relu : 입력값이 0보다 작으면 0으로 출력, 0보다 크면 입력값 그대로 출력하는 유닛

In [ ]:
class Net(nn.Module):

    def __init__(self):
        # 부모 클래스인 nn.Module 의 __init__ 매서드를 Net 에서도 그대로 사용하겠다
        super(Net, self).__init__()
        
        # 5개 층으로 이루어진 모델 생성 (input 784, output10)
        self.l1 = nn.Linear(784, 520) 
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)
    
    # forward prapagation
    def forward(self, x):
        
        # -1은 첫번째 차원은 파이토치 자체적으로 설정한다는 의미
        x = x.view(-1, 784)  # Flatten the data (n, 1, 28, 28)-> (n, 784)
        
        # x를 Linear 모델에 각각 넣고 relu함수를 통해 나온 결과값을 x에 저장
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) 

## Net의 인스턴스 선언, 오차함수에 필요한 클래스 생성, 최적화 클래스 생성

In [ ]:
# model : Net class 의 instance
model = Net()

# 모델에서 사용하는 input Tensor들은 input = input.to(device) 을 호출해야함
model.to(device)

# CrossEntropyLoss 
criterion = nn.CrossEntropyLoss()

# model.parameters() : SGD 생성자에 model.parameters()를 호출하면
# 모델의 멤버인 2개의 nn.Linear 모듈의 학습 가능한 매개변수들이 포함됨 (w,b)
# update 과정에서 SGD 알고리즘을 채택 (확률적 경사하강법)
# lr=1e-2 로 바꿔 쓰기 가능
# momentum : momentum factor (default: 0)
# 모멘텀은 경사 하강법에 관성을 더해 주는 것
# 매번 기울기를 구하지만, 가중치를 수정하기전 이전 수정 방향(+,-)를 참고하여 같은 방향으로 일정한 비율만 수정되게 하는 방법
# 수정이 양(+) 방향, 음(-) 방향 순차적으로 일어나는 지그재그 현상이 줄어들고, 
# 이전 이동 값을 고려해여 일정 비율만큼 다음 값을 결정하므로 관성의 효과를 낼 수 있음.
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

## 훈련 함수 정의

In [ ]:
def train(epoch):
    
    # model.train() -> 학습 모드
    # 학습을 계속하고 싶다면, model.train() 을 호출하여 학습 모드로 설정해야 함
    model.train()
    
    # batch_idx에 배치의 인덱스가 들어가고
    # data에 MNIST dataset의 transform(img) 부분이 들어가고
    # target에 MNIST dataset의 target_transfor(target) 부분이 들어감
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device) # 디바이스에 쑤셔넣기
        
        #optimizer.zero_grad() 를 사용하여 수동으로 변화도 버퍼를 0으로 설정하는 것에 유의해야함
        # 이는 역전파(Backprop) 과정에서 변화도가 누적되기 때문
        optimizer.zero_grad()
        
        # y_pred 부분
        # Net class 의 인스턴스 model 에 data를 집어넣어서 return 된 10행짜리의 결과값을 output에 저장
        output = model(data)
        
        # loss 함수 생성_ y_pred 자리에 output이, y 자리에 target이 들어간다
        loss = criterion(output, target)
        
        # loss 값 역전파
        loss.backward()
        
        # 최적화 적용
        optimizer.step()
        
        # batch index가 10의 배수일 때마다 몇 번째 훈련 epoch 인지, 총 몇 개 배치 중 몇 번째 배치인지, 이번 미니배치 훈련에서의 오차값이 몇인지
        if batch_idx % 10 == 0:
            print('Train Epoch: {} | Batch Status: {}/{} ({:.0f}%) | Loss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

## test 함수 정의

In [ ]:
def test():
    
    # model.eval() -> test 모드
    # 추론을 실행하기 전에는 반드시 model.eval() 을 호출하여 드롭아웃 및 배치 정규화를 평가 모드로 설정
    # 이걸 안하면 추론 결과가 일관성 없게 출력됨
    model.eval()

    # test_loss 
    test_loss = 0 # test_loss : 손실 값의 합 0 으로 초기화
    correct = 0 # correct 는 정확도 계산을 위한 tool 이다
    
    # 물론 미니배치 단위로 쪼개지는데,
    # data에 MNIST dataset의 transform(img) 부분이 들어가고
    # target에 MNIST dataset의 target_transfor(target) 부분이 들어감
    for data, target in test_loader:
        data, target = data.to(device), target.to(device) # 디바이스에 쑤셔넣기
        
        # y_pred 부분
        # Net class 의 인스턴스 model 에 data를 집어넣어서 return 된 10행짜리의 결과값을 output에 저장
        output = model(data)
        
        # sum up batch loss
        # 손실 값의 합
        test_loss += criterion(output, target).item()
        
        # get the index of the max
        # 최대값의 index return
        # keepdim : 벡터 차원의 유지 여부 결정
        pred = output.data.max(1, keepdim=True)[1]
        
        # view_as : view와 비슷 괄호 안에 들어가는 변수와 tensor size 일치시킴
        # 일치하면 +시킴
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        
    test_loss /= len(test_loader.dataset)
    print(f'===========================\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
          f'({100. * correct / len(test_loader.dataset):.0f}%)')

## 결과 확인 하는 메인함수

In [ ]:
if __name__ == '__main__':
    since = time.time()
    for epoch in range(1, 10):
        epoch_start = time.time() # 시작 시간 저장
        
        train(epoch) # 훈련 시작
        m, s = divmod(time.time() - epoch_start, 60) # divmod -> 훈련 시간을 60 으로 나눈 몫과 나머지를 m과 s 에저장
        print(f'Training time: {m:.0f}m {s:.0f}s') # 훈련한 시간 : m,s 출력
        
        test() #  테스트 시작
        m, s = divmod(time.time() - epoch_start, 60) # divmod -> 테스트 시간을 60 으로 나눈 몫과 나머지를 m과 s 에저장
        print(f'Testing time: {m:.0f}m {s:.0f}s') # 테스트 시간 : m,s 출력

        
    # 총 걸린 시간
    m, s = divmod(time.time() - since, 60)
    print(f'Total Time: {m:.0f}m {s:.0f}s\nModel was trained on {device}!')